In [1]:
import awkward
import numba

In [3]:
awkward.ArrayBuilder().snapshot()

<Array [] type='0 * unknown'>

In [5]:
dir(awkward.ArrayBuilder())

['List',
 'Record',
 'Tuple',
 '_Nested',
 '__abstractmethods__',
 '__array__',
 '__arrow_array__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_attrs',
 '_behavior',
 '_layout',
 '_repr',
 '_wrap',
 'append',
 'attrs',
 'begin_list',
 'begin_record',
 'begin_tuple',
 'behavior',
 'boolean',
 'bytestring',
 'complex',
 'datetime',
 'end_list',
 'end_record',
 'end_tuple',
 'extend',
 'field',
 'index',
 'integer',
 'list',
 'null',
 'numba_type',
 'real',
 'record',
 'show',
 'snapshot',
 'string',
 'timedelta',
 'to_list',
 'to_numpy',
 'tolist',
 'tuple',
 'type',
 'typestr']

In [7]:
a = awkward.Array(
    [
        [1.1,2,3],
        [2],
        [],
        [4,5]
    ]
)
a

<Array [[1.1, 2, 3], [2], [], [4, 5]] type='4 * var * float64'>

In [14]:
@numba.njit
def explode_numba_wrap(ev, builder):
    for e in ev:
        builder.begin_list()
        n = len(e)
        for i in range(n):
            builder.begin_list()
            for j in e:
                builder.real(j)
            builder.end_list()
        builder.end_list()
    return builder
                

def explode(ev):
    if awkward.backend(ev) == "typetracer":
        # here we fake the output of numba wrapper function since
        # operating on length-zero data returns the wrong layout!
        awkward.typetracer.length_zero_if_typetracer(ev) # force touching of the necessary data
        return awkward.Array(awkward.Array([]).layout.to_typetracer(forget_length=True))
    return explode_numba_wrap(ev, awkward.ArrayBuilder()).snapshot()

In [15]:
explode(a)

<Array [[[1.1, 2, 3], ..., [1.1, ..., 3]], ...] type='4 * var * var * float64'>